### Настройка окружения

Для начала вам необходимо выполнить ряд команд чтобы настроить окружение для дальнейшей работы, это позволит первое время не заниматься долгим деплоем, а сразу начать писать код и работать с airflow.

In [1]:
# Установка Airflow
# Установка Airflow
!pip install apache-airflow==2.1.4
!pip install apache-airflow-providers-http
!pip install apache-airflow-providers-telegram

!airflow db init

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 20.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 48.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.7/155.7 KB 8.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.5/210.5 KB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 98.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 KB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.4/61.4 KB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.0/239.0 KB 29.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.7/133.7 KB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.0/47.0 KB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━

In [2]:
# Создадим папку dags
# В этой папке лежат скрипты для создания дагов
# Это стандартное имя для  данной папки
!mkdir /root/airflow/dags
!mkdir /root/airflow/plugins
!mkdir /root/airflow/plugins/custom_plugin
!mkdir /root/airflow/plugins/custom_plugin/operators
!mkdir /root/airflow/plugins/custom_plugin/hooks
!mkdir /root/airflow/plugins/custom_plugin/sensors
!touch /root/airflow/dags/dag.py
!touch /root/airflow/dags/telegram.py
!touch /root/airflow/plugins/custom_plugin/__init__.py
!touch /root/airflow/plugins/custom_plugin/operators/__init__.py
!touch /root/airflow/plugins/custom_plugin/operators/custom_operator.py
!touch /root/airflow/plugins/custom_plugin/hooks/__init__.py
!touch /root/airflow/plugins/custom_plugin/sensors/__init__.py

In [3]:
!airflow users create \
          --username admin \
          --firstname admin \
          --lastname admin \
          --role Admin \
          --email admin@example.org \
          -p 12345

[2023-02-27 18:25:02,498] {manager.py:243} INFO - Inserted Role: Admin
[2023-02-27 18:25:02,509] {manager.py:243} INFO - Inserted Role: Public
[2023-02-27 18:25:02,512] {manager.py:824} WARNING - No user yet created, use flask fab command to do it.
[2023-02-27 18:25:02,585] {manager.py:504} INFO - Created Permission View: can edit on Passwords
[2023-02-27 18:25:02,604] {manager.py:562} INFO - Added Permission can edit on Passwords to role Admin
[2023-02-27 18:25:02,630] {manager.py:504} INFO - Created Permission View: can read on Passwords
[2023-02-27 18:25:02,650] {manager.py:562} INFO - Added Permission can read on Passwords to role Admin
[2023-02-27 18:25:02,691] {manager.py:504} INFO - Created Permission View: can edit on My Password
[2023-02-27 18:25:02,703] {manager.py:562} INFO - Added Permission can edit on My Password to role Admin
[2023-02-27 18:25:02,718] {manager.py:504} INFO - Created Permission View: can read on My Password
[2023-02-27 18:25:02,730] {manager.py:562} INFO 

### Практика

#### Первый плагин

Поместите в /root/airflow/plugins/custom_plugin/operators/custom_operator.py следующий код

```python
from airflow.models.baseoperator import BaseOperator

class HelloOperator(BaseOperator):

    def __init__(
            self,
            name: str,
            **kwargs) -> None:
        super().__init__(**kwargs)
        self.name = name

    def execute(self, context):
        message = "Hello {}".format(self.name)
        print(message)
        return message
```



Поместите в dag.py следующий код

**Задание #1**

Сообщение в логе будет ответом к задаче.

```python
from custom_plugin.operators.custom_operator import HelloOperator
from airflow import DAG
from datetime import timedelta, datetime
from airflow.utils.dates import days_ago
 
# Создадим объект класса DAG
dag = DAG(
    'dag',
    start_date=datetime(2015, 12, 1),
    schedule_interval='@daily')
hello_task = HelloOperator(task_id='sample-task', name='foo_bar', dag=dag)
```

#### Нотификация

Вставьте данный код в dag.py

**Задание #2**

Сообщение в телеграм будет ответом.

```python
from airflow import DAG
from datetime import timedelta, datetime
from airflow.utils.dates import days_ago
from custom_plugin.operators.custom_operator import HelloOperator
from airflow.providers.telegram.operators.telegram import TelegramOperator

 
def on_success_callback(context):
    send_message = TelegramOperator(
        task_id='send_message_telegram',
        telegram_conn_id='telegram_id',
        chat_id='-1001525736146',
        text='Hello from Airflow!',
        dag=dag)
    return send_message.execute(context=context)

# Создадим объект класса DAG
dag = DAG(
    'dag',
    start_date=datetime(2015, 12, 1),
    on_success_callback=on_success_callback,
    schedule_interval='@daily')
hello_task = HelloOperator(task_id='sample-task', name='foo_bar', dag=dag)
```

Создайте чат и бота в телеграмме, добавьте бота в чат выдам админские права, создайте **connection** (с именем telegram_id) со следующими параметрами


- Password: API Token который можно получить через https://telegram.me/BotFather
- Host: ID вашего чата, можно получить добавив в чат @raw_data_bot и написать /my_id
- Connection Type: HTTP


In [4]:
# Перезапустим шедулер
# Это должно ускорить поиск нашего дага
# Нужно подождать пару минут пока даг появится
!airflow scheduler -D

  ____________       _____________
 ____    |__( )_________  __/__  /________      __
____  /| |_  /__  ___/_  /_ __  /_  __ \_ | /| / /
___  ___ |  / _  /   _  __/ _  / / /_/ /_ |/ |/ /
 _/_/  |_/_/  /_/    /_/    /_/  \____/____/|__/


In [5]:
# Запуск веб сервера
!airflow webserver -p 18273 -D

  ____________       _____________
 ____    |__( )_________  __/__  /________      __
____  /| |_  /__  ___/_  /_ __  /_  __ \_ | /| / /
___  ___ |  / _  /   _  __/ _  / / /_/ /_ |/ |/ /
 _/_/  |_/_/  /_/    /_/    /_/  \____/____/|__/
Running the Gunicorn Server with:
Workers: 4 sync
Host: 0.0.0.0:18273
Timeout: 120
Logfiles: - -
Access Logformat: 
[2023-02-27 18:26:56,409] {dagbag.py:538} INFO - Filling up the DagBag from /dev/null


In [6]:
# Последующие команды не имеют отношения к Airflow
# Они нужни только для корректной работы веб морды
# в среде Google Colab

!pip install pyngrok
!ngrok authtoken 2MBRgJ066swk4ThgvdLuWtytQAI_3WLNSDpTtkwHYi33C27nr # найти его можно https://dashboard.ngrok.com/get-started/setup 

# Эта команда просто отображет веб морду на другой адрес
# Его вы можете найти https://dashboard.ngrok.com/cloud-edge/status
# При каждом отключении ссылка будет меняться
!nohup ngrok http -log=stdout 18273 > /dev/null &

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 761.3/761.3 KB 14.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-5.2.1-py3-none-any.whl size=19792 sha256=473172b970b828f934fcdd8128a69d755b14334b1dc26e0650497d3467fd1e27
  Stored in directory: /root/.cache/pip/wheels/5d/f2/70/526da675d32f17577ec47ac4c663084efe39d47c826b6c3bb1
Successfully built pyngrok
Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml
nohup: redirecting stderr to stdout
